In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [2]:
itPostsRows = sc.textFile("../Data/Italian_Stack_Exchange/italianPosts.csv")
itPostsSplit = itPostsRows.map(lambda x: x.split("~"))
from pyspark.sql import Row
from datetime import datetime
def toIntSafe(inval):
  try:
    return int(inval)
  except ValueError:
    return None

def toTimeSafe(inval):
  try:
    return datetime.strptime(inval, "%Y-%m-%d %H:%M:%S.%f")
  except ValueError:
    return None

def toLongSafe(inval):
  try:
    return long(inval)
  except ValueError:
    return None
    
def stringToPost(row):
  r = row.encode('utf8').split("~")
  return Row(
    toIntSafe(r[0]),
    toTimeSafe(r[1]),
    toIntSafe(r[2]),
    r[3],
    toIntSafe(r[4]),
    toTimeSafe(r[5]),
    toIntSafe(r[6]),
    toIntSafe(r[7]),
    r[8],
    toIntSafe(r[9]),
    toLongSafe(r[10]),
    toLongSafe(r[11]),
    long(r[12]))
from pyspark.sql.types import *
postSchema = StructType([
  StructField("commentCount", IntegerType(), True),
  StructField("lastActivityDate", TimestampType(), True),
  StructField("ownerUserId", LongType(), True),
  StructField("body", StringType(), True),
  StructField("score", IntegerType(), True),
  StructField("creationDate", TimestampType(), True),
  StructField("viewCount", IntegerType(), True),
  StructField("title", StringType(), True),
  StructField("tags", StringType(), True),
  StructField("answerCount", IntegerType(), True),
  StructField("acceptedAnswerId", LongType(), True),
  StructField("postTypeId", LongType(), True),
  StructField("id", LongType(), False)
  ])
rowRDD = itPostsRows.map(lambda x: stringToPost(x))
itPostsDFStruct = sqlContext.createDataFrame(rowRDD, postSchema)

from pyspark.sql.functions import *

In [3]:
itPostsDFStruct.show()

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+-----+--------------------+-----------+----------------+----------+----+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+-----+--------------------+-----------+----------------+----------+----+
|           4|2013-11-11 18:21:...|         17|&lt;p&gt;The infi...|   23|2013-11-10 19:37:...|     null| null|                    |       null|            null|         2|1165|
|           5|2013-11-10 20:31:...|         12|&lt;p&gt;Come cre...|    1|2013-11-10 19:44:...|       61| null| &lt;word-choice&gt;|          1|            null|         1|1166|
|           2|2013-11-10 20:31:...|         17|&lt;p&gt;Il verbo...|    5|2013-11-10 19:58:...|     null| null

In [8]:
def stringToVote(r):
  return Row(
    long(r[0]),
    long(r[1]),
    int(r[2]),
    toTimeSafe(r[3])
    )
voteSchema = StructType([
    StructField("id", LongType(), False),
    StructField("postId", LongType(), False),
    StructField("voteTypeId", IntegerType(), False),
    StructField("creationTime", TimestampType(), False),
    ])

itVotesRows = sc.textFile("../Data/Italian_Stack_Exchange/italianVotes.csv")
itVotesSplit = itVotesRows.map(lambda x : x.encode('utf8').split("~"))
rowRDD = itVotesSplit.map(lambda x: stringToVote(x))
itVotesDFStruct = sqlContext.createDataFrame(rowRDD, voteSchema)

In [16]:
itPostsDFStruct.join(itVotesDFStruct.withColumnRenamed('id','voteId'), itPostsDFStruct.id==itVotesDFStruct.postId,'inner').show()

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+-----+--------------------+-----------+----------------+----------+----+------+------+----------+-------------------+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|voteId|postId|voteTypeId|       creationTime|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+-----+--------------------+-----------+----------------+----------+----+------+------+----------+-------------------+
|           1|2013-11-06 00:22:...|         18|&lt;p&gt;&quot;De...|    3|2013-11-06 00:05:...|     null| null|                    |       null|            null|         2|  26|    62|    26|         2|2013-11-06 00:00:00|
|           1|2013-11-06 00:22:...|         18|&lt;p&gt;&quot;De...|    3|2013-11-06 00:05:...|     null| nu